---

# CSCI 3202, Fall 2022
# Homework 4
# Due: Friday September 30, 2022 at 6:00 PM

<br> 

### Your name: Vincent Curran

<br> 

---

Some useful packages and libraries:




In [1]:
from scipy import stats
import unittest
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

---

# Hidden Markov Models
In this assignment, you will build a Hidden Markov Model from a data set to detect typos in text. One major difference between this assignment and previous assignments in this class is we're not providing any skeleton code, you need to generate your code from scratch. You can use any code provided during lecture or on previous assignments. 
<p><strong>Please read the entire assignment before beginning.</strong> 
    
<p>The main components of the assignment are the following:

### Part I - Building an HMM from data
1.  Implement a method to parse the data file and build an HMM from the data.
2.  Implement a method, Laplace smoothing, to smooth the transition and emission probabilities

### Part II - Implement the Viterbi algorithm
1.	Implement the Viterbi algorithm for finding the most likely sequence of states through the HMM, given "evidence"; and
2.	Run your code on a separate dataset and explore its performance.
    
### Part III - Summarize your results
    



## Part 1 - Building an HMM from data
The first part of the assignment is to build an HMM from data. Recall that an HMM involves a hidden state that changes over time, as well as observable evidence that is used to infer the hidden state. In our example from lecture, we used the observation of how a patient felt to infer whether they were healthy or sick. 

An HMM is defined by three sets of probabilities:
1.	For each state s, the probability of observing each output o at that state (P(E[t]=o | X[t]=s). These are the emission probabilities.
2.	From each state s, the probability of traversing to every other state s' in one time step (P(X[t+1]=s' | X[t]=s)). These are the transition probabilities.
3.	A distribution over the start state (P(X[0])). This is the initial state distribution.

For the start state distribution (P(X[0])), in this assignment, you will assume that there is a single dummy start state, distinct from all other states, and to which the HMM can never return.  When you implement the Viterbi algorithm in Part II, you will need to include the probability of making a transition from this dummy start state to each of the other states.

### Generate the conditional probability tables
To generate the transition and emission probability tables, use the data file called <em>typos20.data</em> on Canvas that contains sequences of state-output pairs, i.e., sequences of the form: <br>

x[1] e[1] <br>
x[2] e[2] <br>
. <br>
. <br>
. <br>
x[n] e[n]. <br>  

In this data, the x[i] is the state and the e[i] is the observation at time i. 

For instance, to estimate the probability of output o being observed in state s, you count the number of times that output o appears with state s in the given data, and divide by a normalization constant (so that the probabilities of all outputs from that state add up to one).  In this case, that normalization constant would simply be the number of times that state s appears at all in the data.

In this problem, state is letter that should have been typed, and output is the letter that was typed.  Given a sequence of outputs (i.e., actually typed letters), the problem is to reconstruct the hidden state sequence (i.e., the intended sequence of letters).  Thus, data for this problem looks like this:

i i <br>
n n <br>
t t <br>
r r <br>
o o <br>
d x <br>
u u <br>
c c <br>
t t <br>
i i <br>
o i <br>
n n <br>
_ _ <br>
t t <br>
h h <br>
e e <br>
_ _ <br>

where the left column is the correct text and the right column contains text with errors.

### Data
Data for this assignment was generated by starting with a text document, in this case, the Unabomber's Manifesto, which was chosen not for political reasons, but for its convenience being available on-line and of about the right length, converting all numbers and punctuation to white space and converting all letters to lower case.  The remaining text is a sequence only over the lower case letters and the space character, represented in the data files by an underscore character.  Next, typos were artificially added to the data as follows: with 80% probability, the correct letter is transcribed, but with 20% probability, a randomly chosen neighbor (on an ordinary physical keyboard) of the letter is transcribed instead.  Space characters are always transcribed correctly. 

As an example, the original document begins:

<em>introduction the industrial revolution and its consequences have been a disaster for the human race they have greatly increased the life expectancy of those of us who live in advanced countries but they have destabilized society have made life unfulfilling have subjected human beings to indignities have led to widespread psychological suffering in the third world to physical suffering as well and have inflicted severe damage on the natural world the continued development of technology will worsen the situation it will certainly subject human beings to greater indignities and inflict greater damage on the natural world it will probably lead to greater social disruption and psychological suffering and it may lead to increased physical suffering even in advanced countries the industrial technological system may survive or it may break down if it survives it may eventually achieve a low level of physical and psychological suffering but only after passing through a long and very painful period of adjustment and only at the cost of permanently reducing human beings and many other living organisms to engineered products and mere cogs in the social machine </em>

With 20% noise, it looks like this:

<em>introductipn the industfial revolhtjon and its consequences bafw newn a diszster rkr the yumab race thdy have grwatky increased the ljte esoectandy od thosr of is who libe in advanced coubfries but they have fewtabipuzee xociwty have made life ujfuorillkng have wubjwdted humah beints to incihbjtids have led to qidespreze lsyxhllotical shffeding kn tne third wkrld to phyxicql sufcefimg as weol and hqve ingoidtex srvere damsge on the natural world the confinued developmeng of twvhjllogy will wotsen thd situation it wull certaknly sunjrct yyman beingw tl greater ibdignities snd infpixt greagwr damsge on fhe natural alrld it wjlk probably lwad tk grezter sofiqp disrupgiln and pstchokofucal wufterkng anc it may kead fl uncreqxed pgusiczl sucfreinh even in acgajved countries the indhsteial tedhnologicak system may survivr or ut nay brezk down uf it survives it nay evenyuakly achieve a los lwvel of phyxkcal and psycyoligical sufveribg but only after passing theough a long amd very painful periox od adjuwtmebt and only at the fost kf permsnently reducing hymaj veings abs nsjy otgwr kuving orbanisms to envineered leoduxfs amd mere clgs in thr soxiap maxhjne </em>

The error rate (fraction of characters that are mistyped) is about 16.5% (less than 20% because space characters were not corrupted).

### Implement Laplace smoothing
When you generate the transition and emission probabilities, you will also need to smooth the estimates.  Here's the rationale, consider flipping a coin for which the probability of heads is p, where p is unknown, and our goal is to estimate p. We count how many times the coin comes up heads and divide by the total number of coin flips.  If we flip the coin 1000 times and it comes up heads 367 times, it is very reasonable to estimate p as approximately 0.367.  However, suppose we flip the coin only twice and we get heads both times.  Is it reasonable to estimate p as 1.0?  Intuitively, given that we only flipped the coin twice, we don’t have enough data to conclude that the coin will always come up heads. Smoothing is a way of avoiding these types of conclusions when we have limited data available.  

A simple smoothing method, called Laplace smoothing (or Laplace's law of succession or add-one smoothing in your textbook), is to estimate p by:

<p>
<center>$\frac{(1+number of heads)}{(2+number of flips)}$</center>

We add 1 to each output that could be observed in the numerator, and add the total number of states to the denominator.

If we are keeping count of the number of heads and the number of tails, this rule is equivalent to starting each of our counts at one, rather than zero.  This latter view generalizes to the case in which there are more than two possible outcomes (for instance, estimating the probability of a die coming up on each of its six faces). 
Another advantage of Laplace smoothing is that it avoids estimating any probabilities to be zero, even for events never observed in the data.  For HMMs, this is important since zero probabilities can be problematic for some algorithms.
    
For this assignment, you should use Laplace-smoothed estimates of probabilities.  For instance, returning to the problem of estimating the probability of output o being observed in state s, you would use one plus the number of times output o appears in state s in the given data, divided by a normalization constant. In this case, the normalization constant would be the number of times state s appears in the data, plus the total number of possible outputs. You will need to also work out Laplace-smoothed estimates for item 2, i.e., for the probability of making a transition from one state to another, as well as the probability of making a transition from the dummy start state to any of the other states. 


In [2]:
## Your code needs to display the first 10 lines of your emission and transition tables, 
## and generate an output file with the complete tables, called Assignment8Output_Lastname_Firstname
## Your code also needs to be structured as an HMM class
class HMM:
    def __init__(self):
        # List of all lowercase chars, and 'space'
        self.all_states = list(map(chr, range(97, 123))) + ['_']

        # Using equal chance to start in each state, instead of a dummy state
        self.initial_p = {char: 1/len(self.all_states) for char in self.all_states}
        
        self.so = []           # list of (state, output) pairs
        self.expected = {}     # dict of how many times each char is expected
        self.emissions = {}    # dict of emission probabilities
        self.transitions = {}  # dict of transition probabilities 
        
        # Populate the HMM with data
        self.readData()
        self.countExpected()
        self.countChars()
        self.generateProbabilities()
    
    def readData(self):
        # Read data file as list of (state/output) pairs
        with open('typos20.data', 'r') as file:
            lines = file.readlines()

        for x in lines:
            (state, output) = x.split(' ')
            output = output[:len(output)-1] # remove newline char
            self.so.append((state, output)) 

    def countExpected(self):
        # Count # of times each character is expected to appear
        expected = {char: 0 for char in self.all_states}

        for pair in self.so:
            expected[pair[0]] = expected[pair[0]]+1

        self.expected = expected

    def countChars(self, state=None):
        # Count # of times each character appears
        # If state is given, count # of times each char appears
        # when in that state
        totals = {char: 0 for char in self.all_states}

        for pair in self.so:
            if state is None or pair[0] == state:
                totals[pair[1]] = totals[pair[1]]+1
      
        if state is None:
            self.totals = totals
        return totals

    def generateProbabilities(self):
        # Generate 'emission' and 'transition' dictionaries (matrices)
        # Get smooth probability of each action from each state (emission)
        # Get smooth probability of transitioning to each state from each state (transitions)
        def smoothProbability(char, counts):
            # Generate a Laplace smoothed dictionary of probabilities
            return {x: (counts[x]+1)/(self.expected[char]+len(self.all_states)) for x in counts}
                
        for char in self.all_states:
            self.emissions[char] = smoothProbability(char, self.countChars(char))
            self.transitions[char] = smoothProbability(char, self.countNextStates(char))        
        
    def countNextStates(self, state):
        # Count total # of times each state appears after a given state
        totals = {char: 0 for char in self.all_states}
        for i in range(len(self.so)-1):
            if self.so[i][0] == state:
                totals[self.so[i+1][0]] = totals[self.so[i+1][0]]+1
        return totals

    def printPartialTables(self):
        # Print the first 2 elements of emission/transition dicts
        print("Emissions:", {k: hmm.emissions[k] for k in list(hmm.emissions)[:2]}, '\n')
        print("Transitions:", {k: hmm.transitions[k] for k in list(hmm.transitions)[:2]})
    
    def outputTables(self):
        # Generate output file of full emission/transition dicts
        def prettyPrint(d):
            # Return string with each item in dict on new line
            return "{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in d.items()) + "}"
        
        with open("Assignment8Output_Curran_Vincent", 'w') as file:
            file.write("Emissions:\n" + prettyPrint(hmm.emissions)
                       + "\nTransitions:\n" + prettyPrint(hmm.transitions))

In [3]:
# Helper methods
def printTotalProbabilities(dct):
    # To check probabilities add to one
    for char in dct:
        total = sum([prob for prob in dct[char].values()])
        print(total)

def printErrorRate(so):
    # Check error rate of a given list of (state, output) pairs
    total = len(so)
    incorrect = sum([1 for (state, output) in so if state != output]) 
    print("{:.2f}%".format((incorrect/total) * 100))
    
def printReadableOutput(so, st=None):
    # Print the outputs from a (state, output) pair as one string
    # Passing anything to st makes the states print instead of outputs
    t = ""
    for (s, o) in so:
        t = t + s if st is not None else t + o
    print(t)

In [4]:
# # Test Outputs
# hmm = HMM()

# print("State-Output List", hmm.so)
# print("Totals (Expected):", hmm.expected)
# print("Totals (Actual):", hmm.totals)
# print("Emissions:", hmm.emissions)
# print("Transitions:", hmm.transitions)

# printTotalProbabilities(hmm.transitions)
# printTotalProbabilities(hmm.emissions)

# printErrorRate(hmm.so)
# printReadableOutput(hmm.so, True)

In [5]:
# Create HMM, print partial emission/transition tables, and output full tables to file
hmm = HMM()
hmm.printPartialTables()
hmm.outputTables()

Emissions: {'a': {'a': 0.794791561522156, 'b': 0.0001009387301907742, 'c': 0.0001009387301907742, 'd': 0.0001009387301907742, 'e': 0.0001009387301907742, 'f': 0.0001009387301907742, 'g': 0.0001009387301907742, 'h': 0.0001009387301907742, 'i': 0.0001009387301907742, 'j': 0.0001009387301907742, 'k': 0.0001009387301907742, 'l': 0.0001009387301907742, 'm': 0.0001009387301907742, 'n': 0.0001009387301907742, 'o': 0.0001009387301907742, 'p': 0.0001009387301907742, 'q': 0.06823458160896335, 'r': 0.0001009387301907742, 's': 0.068840213990108, 't': 0.0001009387301907742, 'u': 0.0001009387301907742, 'v': 0.0001009387301907742, 'w': 0.0001009387301907742, 'x': 0.0001009387301907742, 'y': 0.0001009387301907742, 'z': 0.06581205208438477, '_': 0.0001009387301907742}, 'b': {'a': 0.0005065856129685917, 'b': 0.7988855116514691, 'c': 0.0005065856129685917, 'd': 0.0005065856129685917, 'e': 0.0005065856129685917, 'f': 0.0005065856129685917, 'g': 0.04609929078014184, 'h': 0.04356636271529889, 'i': 0.0005065


## Part II - Viterbi - finding the most likely sequence

Once your CPTs for your HMM are built, the second part of the assignment is to implement the Viterbi algorithm and use your CPTs to compute the most probable sequence of states (according to the HMM that you built from the data) for a given sequence of outputs.

There is another file on Canvas called *typos20Test.data* that contains test sequences of state-output pairs. You will provide your Viterbi code with just the output part of each of these sequences, and from this, you must compute the most likely sequence of states to produce the output sequence. The state part of these sequences is provided so that you can compare the estimated state sequences generated by your code to the actual state sequences that generated this data.  Note that these two sequences will not necessarily be identical, even if you have correctly implemented the Viterbi algorithm.

A numerical tip: the Viterbi algorithm involves multiplying many probabilities together.  Since each of these numbers is smaller than one (possibly much smaller), you can end up working with numbers that are tiny enough to be computationally indistinguishable from zero. To avoid this problem, it is recommended that you work with log probabilities. To do so, rather than multiplying two probabilities p and q, simply add their logarithms using the rule:
<p>
<center> log(pq) = log(p) + log(q) </center>

You will probably find that there is never a need to store or manipulate actual probabilities; instead, everything can be done using log probabilities.
The text reconstructed using an HMM with the Viterbi algorithm looks like this:
    <p>
*introduction the industrial revoluthon and its consequences bare neen a dissster ror the tuman race they have greatly increased the lite esoectandy od those of is who libe in advanced counfries but they have festabupusee cocisty have made live intiorilling have wibjested human beints to incingitids have led to widesprese lsysullotical suffeding in the third world to physical surcefing as weol and have ingoistes severe damage on the natural world the continued developmeng of techillogy will wotsen the situation it will certaknly sunirct tyman beinge tl greater indithities and infoist greager damage on the natural aleld it will probably owad to grester sofial distuption and pstchomofucal wiftering and it may kead fl increqxed ogusical suctreing even in achanved countries the industeial technologicak system may survive or ut nay break down if it survives it nay eventually achieve a los level of physical and psycholigical survering but only arter passing theough a long and very paindul perios od adjustment and only at the fost of permanently reducing human veings ans nany other kiving organisms to envineered leodusts and mere clys in the social machine*
        
The error rate has dropped to about 10.4%.


In [6]:
def viterbi(hmm, so, input_id=1):
    # hmm : the Hidden Markov Model
    # so : the list of (state, output) to feed to the algorithm (only output is used by default)
    # input_id : controls wether we feed the states or outputs to the algorithm (testing purposes)
    #     Default value of 1 means the outputs are fed
    # Use log probabilities throughout so we don't have to worry about extremely small values
    V = [{}]

    # Calculate probability of 1st char from initial probability and emissions
    for state in hmm.all_states:
        V[0][state] = {'prob': np.log(hmm.initial_p[state]) + np.log(hmm.emissions[state][so[0][input_id]]),
                       'prev': None}

    # Loop through all the observations, creating dictionaries of probabilities and most likely
    # previous states for each time interval (each character in the text)
    for i in range(1, len(so)):
        V.append({})

        for state in hmm.all_states:
            # For each state, loop through all possible states to find the one with
            # the highest probability of transitioning to current state
            # By multiplying previous states prob with the transition to this
            max_transition_p = V[i-1][hmm.all_states[0]]['prob'] + np.log(hmm.transitions[hmm.all_states[0]][state])
            best_prev = hmm.all_states[0]

            for prev in hmm.all_states[1:]:
                if not (so[i][input_id] != '_' and state == '_'):
                    transition_p = V[i-1][prev]['prob'] + np.log(hmm.transitions[prev][state])
                    if transition_p > max_transition_p:
                        max_transition_p = transition_p
                        best_prev = prev
            # Save V value for current 'time' and state by multiplying probability
            # of the highest transition, with the emission prob for the current state and output
            V[i][state] = {'prob': max_transition_p + np.log(hmm.emissions[state][so[i][input_id]]),
                           'prev': best_prev}

    path = []
    prob = 0.0
    best_state = None
    # Find last state w/ highest prob
    for state, data in V[-1].items():
        if data['prob'] < prob:
            prob = data['prob']
            best_state = state

    path.append(best_state)
    prev = best_state

    # Backtrack through all the previous states that got us there
    for i in range(len(V)-2, -1, -1):
        path.insert(0, V[i+1][prev]['prev'])
        prev = V[i+1][prev]['prev']

    return path

In [7]:
# Read test output file
input_so = []

with open('typos20Test.data', 'r') as file:
    lines = file.readlines()[1:] # remove the '..' line at the beginning 

for x in lines:
    (state, output) = x.split(' ')
    output = output[:len(output)-1] # remove newline char
    input_so.append((state, output)) 

# Run Viterbi with the HMM and test file outputs
opt = viterbi(hmm, input_so)

# Format output 
est = tuple(zip([s for s, o in input_so], opt))

# Print error rates and produced text
print("Original Error Rate:")
printErrorRate(input_so)
# printReadableOutput(input_so) # print the original output
print("New Error Rate:")
printErrorRate(est)
print("Reconstructed Text:")
printReadableOutput(est)

Original Error Rate:
16.14%
New Error Rate:
10.44%
Reconstructed Text:
introduction_the_industrial_revoluthon_and_its_consequences_bare_neen_a_dissster_ror_the_tuman_race_they_have_greatly_increased_the_lite_esoectandy_od_those_of_is_who_libe_in_advanced_counfries_but_they_have_festabupusee_cocisty_have_made_live_intiorilling_have_wibjested_human_beints_to_incingitids_have_led_to_widesprese_lsysullotical_suffeding_in_the_third_world_to_physical_surcefing_as_weol_and_have_ingoistes_severe_damage_on_the_natural_world_the_continued_developmeng_of_techillogy_will_wotsen_the_situation_it_will_certaknly_sunirct_tyman_beinge_tl_greater_indithities_and_infoist_greager_damage_on_the_natural_aleld_it_will_probably_owad_to_grester_sofial_distuption_and_pstchomofucal_wiftering_and_it_may_kead_fl_increqxed_ogusical_suctreing_even_in_achanved_countries_the_industeial_technologicak_system_may_survive_or_ut_nay_break_down_if_it_survives_it_nay_eventually_achieve_a_los_level_of_physical_and_psycholigic

## Part III - Summarize your results
As part of this assignment, include a short report in your notebook describing your work. The report should contain the following:
### Purpose: 
Purpose should be clear. It doesn't need to be long, but it should be clear.
### Procedure:
##### Transition Probabilities
In this section explain the way you have generated the Transition probabilities and the data structure you used.
##### Emission Probabilities
In this section explain the way you have generated the Emission probabilities and the data structure you used.
##### Viterbi Algorithm
In this section explain about the algorithm and the data structure you used for storing the values in the matrix with its back pointer.
### Data
Explain about the data you used, including the data included in the data set and any pre-processing you did to the data.
### Results
Explain your observations, including the sentences that you tested and their corresponding outputs. Were there any sentences where your algorithm did not work?


# Report: 
### Purpose: 
The purpose of this model was to generate the most likely source text given a piece of garbled text (text with typos) using a HMM and the Viterbi algorithm. 
### Procedure:
##### Transition Probabilities
I used a dictionary of dictionaries to hold the emission probabilities. This was so I could map the probabilities from each state to each state (basically a 2D matrix). I generated the probabilities by counting the total number of times each state appeared after a given character and was expected. Then, I divided (the total number of times each char appeared next + 1) over (the times it was expected + the number of states). I added the extra numbers to the numerator and denominator to implement laplace smoothing.
##### Emission Probabilities
I used a dictionary of dictionaries to hold the emission probabilities. This was so I could map the probabilities from each state to each state (basically a 2D matrix). I generated the probabilities by counting the total number of times each state appeared and was expected. Then, I divided (the total number of times each char actually appeared + 1) over (the times it was expected + the number of states). I added the extra numbers to the numerator and denominator to implement laplace smoothing.
##### Viterbi Algorithm
To store the V matrix I used a list of dictionaries. The index in the list represented which 'index' from the input states. The dictionary keys were all the possible states, and the values were: probability of the state, the most likely previous state. This is what allowed me to track backwards through the dict at the end. At a high level, the algorithm uses the chances of specific characters being output from a given sate as well as the probabilities of one state transitioning to another (generated from the initial data file) to determine the most likely sequence of events which occurred, by determining the most likely paths to end with each state. 
### Data
The data was a set of text, with one being the actual text, and the other being what was 'typed' (the garbled text). The first thing I did with the data was convert it to a list of tuples of the form (state, output). In Part 2, I ignored the first line (..) and decided to base the initial probabilities off the emission probabilities, as it seemed simpler than accounting for a 'dummy state'.

### Results
I observed the error rate in the text decrease to ~10.4%. I tested the whole Typos20Test file, and the results can be seen above. One example is: "" got converted to "". Some sentences that didn't work were "" to "" and " " to "", likely because of the amount of typos, unique words, or unique substrings within the words. 
